# Import library

In [2]:
import os
import cv2
import pandas as pd
import seaborn as sns
import numpy as np 
import matplotlib.pyplot as plt

from PIL import Image
from tqdm import tqdm
from joblib import Parallel, delayed

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import StandardScaler

from skimage import io, color
from skimage.feature import hog
from skimage.transform import resize

# Data loading

In [3]:
images_dir = '/kaggle/input/nhapmoncv/data/images'

classes = [d for d in os.listdir(images_dir) if os.path.isdir(os.path.join(images_dir, d))]

label_map = {cls: idx for idx, cls in enumerate(classes)}

data = []
for cls in classes:
    cls_folder = os.path.join(images_dir, cls)
    for fname in os.listdir(cls_folder):
        if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
            file_path = os.path.join(cls_folder, fname)
            label = label_map[cls]
            data.append((file_path, label))

classes = [d.split("-")[-1] for d in os.listdir(images_dir) if os.path.isdir(os.path.join(images_dir, d))]

label_map = {cls: idx for idx, cls in enumerate(classes)}


df = pd.DataFrame(data, columns=['filepath', 'label'])
print(df.head())
print("Number of images:", len(df))
print("Number of classes:", len(classes))

                                            filepath  label
0  /kaggle/input/nhapmoncv/data/images/n02091635-...      0
1  /kaggle/input/nhapmoncv/data/images/n02091635-...      0
2  /kaggle/input/nhapmoncv/data/images/n02091635-...      0
3  /kaggle/input/nhapmoncv/data/images/n02091635-...      0
4  /kaggle/input/nhapmoncv/data/images/n02091635-...      0
Number of images: 20580
Number of classes: 120


In [4]:
label_map = {v:k for k,v in label_map.items()}

In [5]:
df["breed"] = df["label"].map(label_map)

In [6]:
df

,filepath,label,breed
0,/kaggle/input/nhapmoncv/data/images/n02091635-...,0,otterhound
1,/kaggle/input/nhapmoncv/data/images/n02091635-...,0,otterhound
2,/kaggle/input/nhapmoncv/data/images/n02091635-...,0,otterhound
3,/kaggle/input/nhapmoncv/data/images/n02091635-...,0,otterhound
4,/kaggle/input/nhapmoncv/data/images/n02091635-...,0,otterhound
...,...,...,...
20575,/kaggle/input/nhapmoncv/data/images/n02088466-...,119,bloodhound
20576,/kaggle/input/nhapmoncv/data/images/n02088466-...,119,bloodhound
20577,/kaggle/input/nhapmoncv/data/images/n02088466-...,119,bloodhound
20578,/kaggle/input/nhapmoncv/data/images/n02088466-...,119,bloodhound


# Feature extraction using SIFT

In [13]:
sift = cv2.SIFT_create()

filenames = list(df['filepath'])
batch_size = 500
k = 200

sift = cv2.SIFT_create()
kmeans = MiniBatchKMeans(n_clusters=k, batch_size=batch_size, random_state=42)

for img_path in tqdm(filenames, desc="Incremental fitting"):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None: 
        continue
    _, des = sift.detectAndCompute(img, None)
    if des is not None:
        # Sample subset to avoid too many per image
        if len(des) > 200:
            des = des[np.random.choice(len(des), 200, replace=False)]
        kmeans.partial_fit(des)

print("✅ KMeans fitted incrementally.")

Incremental fitting:   0%|          | 0/20580 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
Incremental fitting: 100%|██████████| 20580/20580 [29:23<00:00, 11.67it/s]  

✅ KMeans fitted incrementally.


In [16]:
def get_bovw_vector(des, kmeans):
    hist = np.zeros(k)
    if des is not None:
        words = kmeans.predict(des)
        for w in words:
            hist[w] += 1
    return hist

In [17]:
X = [get_bovw_vector(sift.detectAndCompute(cv2.imread(p, 0), None)[1], kmeans) for p in list(df['filepath'])]
X = StandardScaler().fit_transform(X)

y = np.array(list(df["label"]))

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
# Step 4: Train classifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

RandomForestClassifier()

In [20]:
# Step 5: Evaluate
y_pred = rf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.047619047619047616
